### Importing Libraries

In [2]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

### Reading Files

In [4]:
df=pd.read_csv('cust_churn_dummies.csv')
df.head()

Unnamed: 0  Call  Failure  Complains  Charge  Amount  Seconds of Use  \
0           0              8          0               0            4370   
1           1              0          0               0             318   
2           2             10          0               0            2453   
3           3             10          0               0            4198   
4           4              3          0               0            2393   

   Frequency of SMS  Tariff Plan  Status  Customer Value  Churn  ...  \
0                 5            1       1         197.640      0  ...   
1                 7            1       0          46.035      0  ...   
2               359            1       1        1536.520      0  ...   
3                 1            1       1         240.020      0  ...   
4                 2            1       1         145.805      0  ...   

   Charge  Amount_2  Charge  Amount_3  Charge  Amount_4  Charge  Amount_5  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   
2                 0                 0                 0                 0   
3                 0                 0                 0                 0   
4                 0                 0                 0                 0   

   Charge  Amount_6  Charge  Amount_7  Charge  Amount_8  Charge  Amount_9  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   
2                 0                 0                 0                 0   
3                 0                 0                 0                 0   
4                 0                 0                 0                 0   

   Status_0  Status_1  
0         0         1  
1         1         0  
2         0         1  
3         0         1  
4         0         1  

[5 rows x 39 columns]

In [5]:
df=df.drop('Unnamed: 0', axis=1)

In [6]:
x= df.drop(['Churn', 'Churn_0', 'Churn_1', 'Complains', 'Tariff Plan', 'Status','Charge  Amount'], axis=1)
x.head()

Call  Failure  Seconds of Use  Frequency of SMS  Customer Value  \
0              8            4370                 5         197.640   
1              0             318                 7          46.035   
2             10            2453               359        1536.520   
3             10            4198                 1         240.020   
4              3            2393                 2         145.805   

   Subscription length group_1 - 12  Subscription length group_13 - 24  \
0                                 0                                  0   
1                                 0                                  0   
2                                 0                                  0   
3                                 0                                  0   
4                                 0                                  0   

   Subscription length group_25 - 36  Subscription length group_37 - 48  \
0                                  0                                  1   
1                                  0                                  1   
2                                  0                                  1   
3                                  0                                  1   
4                                  0                                  1   

   Subscription length group_49 - 60  Subscription length group_61 - 72  ...  \
0                                  0                                  0  ...   
1                                  0                                  0  ...   
2                                  0                                  0  ...   
3                                  0                                  0  ...   
4                                  0                                  0  ...   

   Charge  Amount_2  Charge  Amount_3  Charge  Amount_4  Charge  Amount_5  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   
2                 0                 0                 0                 0   
3                 0                 0                 0                 0   
4                 0                 0                 0                 0   

   Charge  Amount_6  Charge  Amount_7  Charge  Amount_8  Charge  Amount_9  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   
2                 0                 0                 0                 0   
3                 0                 0                 0                 0   
4                 0                 0                 0                 0   

   Status_0  Status_1  
0         0         1  
1         1         0  
2         0         1  
3         0         1  
4         0         1  

[5 rows x 31 columns]

In [7]:
y= df['Churn']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Churn, dtype: int64

### Train Test Split

In [9]:
x_train, x_test,y_train, y_test = train_test_split(x,y, random_state=42, test_size=0.2)

## Decision Tree Classifier

In [11]:
model_dt= DecisionTreeClassifier( criterion ='gini', random_state= 42, max_depth= 6, min_samples_leaf= 8)

In [12]:
print(x_train.shape)
print(y_train.shape)


(2520, 31)
(2520,)


In [13]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=42)

In [14]:
y_pred = model_dt.predict(x_test)

In [15]:
model_dt.score(x_test, y_test)

0.8984126984126984

In [16]:
print(classification_report(y_test, y_pred,labels =[0,1]))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       520
           1       0.73      0.66      0.70       110

    accuracy                           0.90       630
   macro avg       0.83      0.81      0.82       630
weighted avg       0.90      0.90      0.90       630



In [17]:
print(metrics.confusion_matrix(y_test, y_pred))


[[493  27]
 [ 37  73]]


As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.
Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.
Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [19]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)

In [20]:
xr_train, xr_test,yr_train, yr_test = train_test_split(X_resampled,y_resampled, test_size=0.2)


In [21]:
model_dt_smote= DecisionTreeClassifier( criterion ='gini', max_depth= 6, min_samples_leaf= 8)


In [22]:
model_dt_smote.fit(xr_train,yr_train)
y_dt_pred_smote = model_dt_smote.predict(xr_test)
model_dt_smote.score(xr_test, yr_test)

print(classification_report(yr_test, y_dt_pred_smote,labels =[0,1]))


              precision    recall  f1-score   support

           0       0.95      0.97      0.96       386
           1       0.97      0.95      0.96       384

    accuracy                           0.96       770
   macro avg       0.96      0.96      0.96       770
weighted avg       0.96      0.96      0.96       770



In [23]:
print(metrics.confusion_matrix(yr_test, y_dt_pred_smote))

[[374  12]
 [ 19 365]]


Now we can see quite better results, i.e. Accuracy: 95 %, and a very good recall, precision & f1 score for minority class.
Let's try with some other classifier.

## Random Forest Classifier

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
model_rf_smote= RandomForestClassifier( n_estimators = 100, random_state =42, criterion = 'gini', min_samples_leaf= 8, max_depth =6)

In [28]:
model_rf_smote.fit(xr_train,yr_train)
y_rf_pred_smote = model_rf_smote.predict(xr_test)
model_rf_smote.score(xr_test, yr_test)

print(classification_report(yr_test, y_rf_pred_smote,labels =[0,1]))


              precision    recall  f1-score   support

           0       0.97      0.94      0.95       386
           1       0.94      0.97      0.95       384

    accuracy                           0.95       770
   macro avg       0.95      0.95      0.95       770
weighted avg       0.95      0.95      0.95       770



In [29]:
print(metrics.confusion_matrix(yr_test, y_rf_pred_smote))

[[363  23]
 [ 13 371]]


## KNN Classifier

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [32]:
knn_model_smote =KNeighborsClassifier()
knn_model_smote.fit(xr_train, yr_train)
y_knn_predict_smote= knn_model_smote.predict(xr_test)
knn_model_smote.score(xr_test, yr_test)

0.9623376623376624

In [33]:
print(classification_report(yr_test, y_knn_predict_smote))

              precision    recall  f1-score   support

           0       0.99      0.94      0.96       386
           1       0.94      0.99      0.96       384

    accuracy                           0.96       770
   macro avg       0.96      0.96      0.96       770
weighted avg       0.96      0.96      0.96       770



In [34]:
print(metrics.confusion_matrix(yr_test, y_knn_predict_smote))

[[362  24]
 [  5 379]]


best accuracy and other scores so far

## Naive Baye's

In [37]:
from sklearn.naive_bayes import GaussianNB

In [38]:
nb_model_smote=  GaussianNB()
nb_model_smote.fit(xr_train, yr_train)
y_nb_predict_smote= nb_model_smote.predict(xr_test)
nb_model_smote.score(xr_test, yr_test)

0.9

In [39]:
print(classification_report(yr_test, y_nb_predict_smote))

              precision    recall  f1-score   support

           0       0.98      0.82      0.89       386
           1       0.84      0.98      0.91       384

    accuracy                           0.90       770
   macro avg       0.91      0.90      0.90       770
weighted avg       0.91      0.90      0.90       770



In [40]:
print(metrics.confusion_matrix(yr_test, y_nb_predict_smote))

[[317  69]
 [  8 376]]


## SVM

In [42]:
from sklearn.svm import SVC

In [43]:
svm_model_smote=  SVC()
svm_model_smote.fit(xr_train, yr_train)
y_svm_predict_smote= svm_model_smote.predict(xr_test)
svm_model_smote.score(xr_test, yr_test)

0.8337662337662337

In [44]:
print(classification_report(yr_test, y_svm_predict_smote))

              precision    recall  f1-score   support

           0       0.89      0.76      0.82       386
           1       0.79      0.90      0.84       384

    accuracy                           0.83       770
   macro avg       0.84      0.83      0.83       770
weighted avg       0.84      0.83      0.83       770



In [45]:
print(metrics.confusion_matrix(yr_test, y_svm_predict_smote))

[[295  91]
 [ 37 347]]


## Logistic Regression

In [47]:
from sklearn.linear_model import LogisticRegression

In [48]:
lr_model_smote=  LogisticRegression(max_iter= 1000, solver='liblinear')
lr_model_smote.fit(xr_train, yr_train)
y_lr_predict_smote= lr_model_smote.predict(xr_test)
lr_model_smote.score(xr_test, yr_test)

0.9428571428571428

In [49]:
print(classification_report(yr_test, y_lr_predict_smote))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94       386
           1       0.93      0.96      0.94       384

    accuracy                           0.94       770
   macro avg       0.94      0.94      0.94       770
weighted avg       0.94      0.94      0.94       770



In [50]:
print(metrics.confusion_matrix(yr_test, y_lr_predict_smote))

[[359  27]
 [ 17 367]]


As we are predicting the customers who will churn we wish to catch as many churn risks as possible

Hence we choose Recall score to choose the best model as it will minimize false negatives. 
Catching a few customers that won't churn doesn't hurt much

KNN classifier has a recall score of 98 for (1 : customer will churn) 

Hence we choose **KNN classifier** as our final model

# Pickling the model

In [53]:
import pickle

In [54]:
filename = 'model.sav'

In [55]:
pickle.dump(knn_model_smote , open(filename, 'wb'))

In [56]:
load_model = pickle.load(open (filename, 'rb'))

In [57]:
model_score_r1= load_model.score(xr_test, yr_test)

In [58]:
model_score_r1

0.9623376623376624

Our final model i.e. KNN Classifier with SMOTEENN, is now ready and dumped in model.sav, which we will use and prepare API's so that we can access our model from UI.